In [35]:
#!/usr/bin/env python
from collections import OrderedDict
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re
import requests
import os

# scrapes any batting table

In [ ]:
class html_table_scraper:
    '''Given a baseball-reference.com url -- scrapes "TEAM BATTING" html table for any season/team into a pandas data frame.
        Allows user to search pandas data frame for a specific player and statistic'''
    
    def scrape_br_html_table(self,url):
        '''Scrapes "TEAM BATTING" html table into pandas data frame'''
        self.url = url

        #create bs object
        r = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(r)

        #find table, find header, find rows
        table = soup.find('div', attrs={'class': 'overthrow table_container'})
        table_head = table.find('thead')
        table_body = table.find('tbody')

        #create table header list
        header = []    
        for th in table_head.findAll('th'):
            key = th.get_text()
            header.append(key)

        #find number of 'empty' rows
        endrows = 0
        for tr in table.findAll('tr'):
            if tr.findAll('th')[0].get_text() in (''):
                endrows += 1

        #find number of rows in table
        rows = len(table.findAll('tr'))
        rows -= endrows + 1  

        #create lists of row data, create ordered dictionary from header and row data...
        #...create list of dictionaries for data frame
        list_of_dicts = []
        for row in range(rows):
            the_row = []
            try:
                table_row = table.findAll('tr')[row]
                for tr in table_row:
                    value = tr.get_text()
                    the_row.append(value)
                od = OrderedDict(zip(header,the_row))
                list_of_dicts.append(od)
            except AttributeError:
                continue 

        #create df
        df = pd.DataFrame(list_of_dicts)

        #change column names to all uppercase for easy searching 
        df.columns = [col.upper() for col in df.columns]

        #strip all bad characters from NAME column to allow searching by name
        df['NAME'] = df['NAME'].str.replace(r'\(([^()]+)\)', '')
        df['NAME'] = df['NAME'].str.replace('*', '')
        df['NAME'] = df['NAME'].str.replace('#', '')

        #set index so it's easier to search
        df1 = df.set_index(keys='NAME')
        return df1

    def search_table(self,url):
        '''given a scraped baseball_reference html table -- searches table for name and statistic of a specific player.
            error handling prompts user to re-enter data if not found in data frame'''
        self.url = url

        df2= self.scrape_br_html_table(url)
        while True:
            try:
                #prompt user for input
                player = input('Enter a player Name:')
                stat = input('Enter a Statistic:') #MUST BE ALL UPPERCASE LETTERS
                #search df
                df3 = df2.loc[player][stat]
                #format output
                d={'Name': player, stat: df3}
                return d
            except KeyError as e:
                print('Error:', e, 'Check spelling')

                pass
            else: 
                break 
table_scraper = html_table_scraper() #instantiate clas

In [22]:
url = 'http://www.baseball-reference.com/teams/BOS/2017.shtml'
table_scraper.scrape_br_html_table(url)

/Applications/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Applications/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


,RK,POS,AGE,G,PA,AB,R,H,2B,3B,...,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB
NAME,,,,,,,,,,,,,,,,,,,,,
Sandy Leon,1,C,28,21,76,74,9,17,3,0,...,.250,.432,.682,81,32,2,0,0,0,1
Mitch Moreland,2,1B,31,35,145,124,17,33,15,0,...,.372,.435,.808,119,54,5,1,0,0,3
Dustin Pedroia,3,2B,33,31,138,119,11,34,6,0,...,.365,.387,.752,105,46,3,0,1,2,1
Xander Bogaerts,4,SS,24,31,131,120,18,42,7,2,...,.405,.442,.846,130,53,3,2,0,0,0
Pablo Sandoval,5,3B,30,17,67,61,7,13,1,0,...,.269,.377,.646,73,23,3,0,0,1,0
Andrew Benintendi,6,LF,22,34,154,136,21,42,7,0,...,.377,.471,.847,129,64,6,2,0,2,1
Jackie Bradley,7,CF,27,20,77,70,5,12,0,1,...,.234,.286,.519,40,20,3,0,0,1,0
Mookie Betts,8,RF,24,32,143,127,18,38,12,0,...,.378,.512,.889,139,65,4,1,0,0,1
Hanley Ramirez,9,DH,33,28,119,105,15,27,3,0,...,.345,.429,.773,109,45,5,0,0,0,0


# Scrape team abbreviations

In [6]:
#use this function to create a list of team names
def format_team_names():
    
    '''returns a list of team name abbreviations used in baseball reference tables'''
    
    names_link = 'http://www.baseball-reference.com/leagues/MLB/2016.shtml'
    df = pd.read_html(names_link, flavor='html5lib', attrs={'class': 'sortable'})

    df_names = df[0]

    teams= df_names['Tm'].tolist()
    del teams[-2:]
    team_names = ['/' + i  for i in teams]
    
    return team_names

In [23]:
team_names = format_team_names()
team_names

['/ARI',
 '/ATL',
 '/BAL',
 '/BOS',
 '/CHC',
 '/CHW',
 '/CIN',
 '/CLE',
 '/COL',
 '/DET',
 '/HOU',
 '/KCR',
 '/LAA',
 '/LAD',
 '/MIA',
 '/MIL',
 '/MIN',
 '/NYM',
 '/NYY',
 '/OAK',
 '/PHI',
 '/PIT',
 '/SDP',
 '/SEA',
 '/SFG',
 '/STL',
 '/TBR',
 '/TEX',
 '/TOR',
 '/WSN']

# Create links for specific team-year combinations

In [24]:
#use this function to create links
def create_links(stem, team_names, years, extension):
    
    '''this function creates links to be iterated over
    str(steem)
       [team_name]
       [year]
       str(extension)'''

    links_lst = []
    for y in years:
        for n in team_names:
            links_lst.append(stem + n + y + extension)
            
    return links_lst

In [26]:
#create specific team-year combinations
stem = 'http://www.baseball-reference.com/teams'
years_lst = ['/2012', '/2013', '/2014', '/2015', '/2016', '/2017']
years_lst2 = ['/2016']
ext = '.shtml'
team_links = create_links(stem, team_names, years_lst2, ext)
team_links

['http://www.baseball-reference.com/teams/ARI/2016.shtml',
 'http://www.baseball-reference.com/teams/ATL/2016.shtml',
 'http://www.baseball-reference.com/teams/BAL/2016.shtml',
 'http://www.baseball-reference.com/teams/BOS/2016.shtml',
 'http://www.baseball-reference.com/teams/CHC/2016.shtml',
 'http://www.baseball-reference.com/teams/CHW/2016.shtml',
 'http://www.baseball-reference.com/teams/CIN/2016.shtml',
 'http://www.baseball-reference.com/teams/CLE/2016.shtml',
 'http://www.baseball-reference.com/teams/COL/2016.shtml',
 'http://www.baseball-reference.com/teams/DET/2016.shtml',
 'http://www.baseball-reference.com/teams/HOU/2016.shtml',
 'http://www.baseball-reference.com/teams/KCR/2016.shtml',
 'http://www.baseball-reference.com/teams/LAA/2016.shtml',
 'http://www.baseball-reference.com/teams/LAD/2016.shtml',
 'http://www.baseball-reference.com/teams/MIA/2016.shtml',
 'http://www.baseball-reference.com/teams/MIL/2016.shtml',
 'http://www.baseball-reference.com/teams/MIN/2016.shtml

# Using output of create_links() -- save links to csv in pwd

In [31]:
#use this function to create csvs
def html_to_csv(links_list):
    
    
    length_list  =len(links_list)
    count = 0

    while count < length_list:
    
        count += 1
        df = table_scraper.scrape_br_html_table(links_list[count])
        file_name = links_list[count][40:48].replace('/', '_')
        df.to_csv('batting_' + file_name + '.csv')
        print(links_list[count])

In [32]:
html_to_csv(team_links)

/Applications/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Applications/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


http://www.baseball-reference.com/teams/ATL/2016.shtml
http://www.baseball-reference.com/teams/BAL/2016.shtml


KeyboardInterrupt: 

# before running all_player_names() make sure you downloaded ALL BATTING TABLES for 1 season so we have a comprehensive list of players

In [ ]:
def all_player_names(fp):
    
    #combine all csvs into 1 giant dataframe
    p = fp
    df_lst = []
    for r,d,f in os.walk(p):
        for n in f:
            fp = os.path.join(r,n)
            df1 = pd.read_csv(fp)
            df_lst.append(df1)

    stats_all = pd.concat(df_lst)
    
    all_player_names = stats_all['NAME'].tolist()
    sorted_names = sorted(all_player_names, key=lambda x: x.split(" ")[-1])
    return sorted_names

In [56]:
file_path = '/Users/Anthony/Desktop/python_projects/twitter/baseball/2016'
player_names = all_player_names(file_path)
player_names

AttributeError: 'float' object has no attribute 'split'

In [42]:
def get_names_from_master_list(master_list, letter):

    new_lst = []
    for i in master_list:
        split = i.split(" ")
        if split[1].startswith(letter):
            new_lst.append(i)
    return new_lst

In [57]:
a_players = get_names_from_master_list(player_names, 'A')
a_players

['Dario Alvarez',
 'Pedro Alvarez',
 'Dariel Alvarez',
 'Jayson Aquino',
 'Erick Aybar']

# Everything below here is related to salary tables

## scrapes salary table and returns it as data frame

In [44]:
def scrape_salary_table(player_url):
    
    page = requests.get(player_url).text


    table_code = page[page.find('<table class="sortable stats_table" id="br-salaries"'):]
    soup = BeautifulSoup(table_code, 'lxml')

    table_body  = soup.find('tbody')
    
    #this block is for salary
    salary_lst = []
    for i in table_body.findAll('tr'):
        for j in i.findAll('td'):
            if j['data-stat'] == 'salary':
                sal = j.get_text()
                salary_lst.append(sal)
                
    #this block is for years              
    years = table_body.findAll('th')                          
    years_lst = []
    for i in years:
        ls = i.get_text()
        years_lst.append(ls)
    del years_lst[-1]
        
    df = pd.DataFrame(years_lst)
    df['salary'] = salary_lst
    df1 = df.reset_index()
    df2 = df1.rename(columns={0: 'years'})
    df3 = df2.set_index(keys = ['years'])
    return df3

In [45]:
l  = 'http://www.baseball-reference.com/players/a/abadfe01.shtml'
scrape_salary_table(l)

,index,salary
years,,
,0,
2011,1,"$418,000"
2012,2,"$485,000"
2014,3,"$525,900"
2015,4,"$1,087,500"
2016,5,"$1,250,000"
2017,6,"$2,000,000"


In [52]:
def salary_to_csv(lst_player_names, br_url_stem):
    
    stem_players = 'http://www.baseball-reference.com'

    br_url_stem = 'http://www.baseball-reference.com/a/'

    r = urllib.request.urlopen(br_url_stem).read()
    soup = BeautifulSoup(r)

    d = {}
    for i in soup.find_all('a'):
        for name in lst_player_names:
            if name in i.text:
                filtered_links = stem_players + i['href']
                #key = filtered_links[44:53]
                #d[key] = name
                df = scrape_salary_table(filtered_links)
                fnames = name.replace(" ", "_")
                df.to_csv(name + '.csv')
                print(name, 'salary csv copied')

In [53]:
br_url_stem = 'http://www.baseball-reference.com/a/'
salary_to_csv(a_players, br_url_stem )

/Applications/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Applications/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Pedro Alvarez
Erick Aybar
Pedro Alvarez
Erick Aybar


KeyboardInterrupt: 

In [ ]:
import shutil